In [ ]:
from flask import Flask, render_template, request, jsonify
import base64
import os
from PIL import Image
import io
import cv2
import tensorflow as tf
import numpy as np

app = Flask(__name__, template_folder='')

# Load all models at startup
model_1 = tf.keras.models.load_model('MNIST-Project-CNN.keras')
model_2 = tf.keras.models.load_model('MNIST-Project-CNN-ver2.keras')
model_3 = tf.keras.models.load_model('MNIST-Project-CNN-ver3.keras')

def prepare(filepath):
    IMG_SIZE = 100  
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) 
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0  

def process_image(image_data):
    # Decode Base64 image data
    image_data_decoded = base64.b64decode(image_data.split(',')[1])
    
    # Create a BytesIO object to handle the image data
    image_stream = io.BytesIO(image_data_decoded)
    
    # Open the image using Pillow
    img = Image.open(image_stream)
    
    # Convert the image to RGB mode if it's not already
    img = img.convert('RGB')
    
    # Get pixel data from the image
    pixels = img.load()
    
    # Loop through each pixel and change black pixels to white
    width, height = img.size
    for x in range(width):
        for y in range(height):
            r, g, b = pixels[x, y]
            if r < 30 and g < 30 and b < 30:
                pixels[x, y] = (255, 255, 255)  # Set pixel to white
    
    img = img.resize((90, 140))
    # Save the modified image to a BytesIO object
    modified_image_stream = io.BytesIO()
    img.save(modified_image_stream, format='JPEG')
    
    # Get the image data from the modified image stream
    modified_image_data = modified_image_stream.getvalue()
    
    return modified_image_data

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    # Get image data and model choice from the request
    image_data = request.form['image']
    model_choice = request.form['model_choice']
    
    if model_choice == 'model 1':
        model = model_1
    elif model_choice == 'model 2':
        model = model_2
    elif model_choice == 'model 3':
        model = model_3
    else:
        return jsonify({'error': 'Invalid model choice'})
    
    # Process the image to turn black pixels to white
    modified_image_data = process_image(image_data)

    # Define the path to save the image with the name "inputimage"
    save_path = os.path.join(app.root_path, 'static', 'images', 'inputimage.jpg')

    # Save the modified image to the specified path
    with open(save_path, 'wb') as f:
        f.write(modified_image_data)
    
    predictions = model.predict(prepare("static/images/inputimage.jpg"))
    predicted_class = np.argmax(predictions, axis=1)
    probability = np.max(predictions) * 100
    print(model_choice)
    print(f'Predicted value: {predicted_class[0]}')
    print(f'Class probabilities: {np.round(predictions[0], decimals=2)}')
    
    # Return a JSON response
    return jsonify({'model_choice': model_choice, 'predicted_class': int(predicted_class), 'probability': round(probability)})

if __name__ == '__main__':
    app.run(debug=False, port=5001)


D:\Anaconda\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [09/Jun/2024 16:08:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2024 16:08:51] "GET /static/images/wallpaper.jpg HTTP/1.1" 304 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


127.0.0.1 - - [09/Jun/2024 16:09:00] "POST /submit HTTP/1.1" 200 -


...

model 3
Predicted value: 6
Class probabilities: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


127.0.0.1 - - [09/Jun/2024 16:15:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2024 16:15:17] "GET /static/images/wallpaper.jpg HTTP/1.1" 304 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


127.0.0.1 - - [09/Jun/2024 16:16:04] "POST /submit HTTP/1.1" 200 -


model 1
Predicted value: 9
Class probabilities: [0.   0.   0.   0.01 0.   0.   0.   0.35 0.   0.63]
